In [81]:
import pandas as pd
import re

In [120]:
def parse_file(nTotalFiles):

    '''Convert WhatsApp chat log text file to a Pandas dataframe.'''

    dfTotal = pd.DataFrame(columns=['timestamp', 'contato', 'status', 'grupo'])

    for nFileNumber in range(1, nTotalFiles+1):

        text_file = 'C:\\Users\\jandi\\Downloads\\drive-download-20210112T214747Z-001\\Conversa do WhatsApp com Intercessão Profética '+ str(nFileNumber) +'.txt'

        # some regex to account for messages taking up multiple lines
        pat = re.compile(r'^(\d\d\/\d\d\/\d\d\d\d.*?)(?=^^\d\d\/\d\d\/\d\d\d\d|\Z)', re.S | re.M)
        with open(text_file, encoding='utf8') as f:
            data = [m.group(1).strip().replace('\n', ' ') for m in pat.finditer(f.read())]

        lContato = []; lDatas = []; lContatoStatus = []; lGrupo = []

        for row in data:

            # timestamp is before the first dash
            lDatas.append(row.split(' - ')[0])

            # sender is between am/pm, dash and colon
            rowFone = row.split(' - ')[1]
            try:
                Fone = re.findall('[0-9]+', rowFone)
                Fone = '+'+Fone[0]+' '+Fone[1]+' '+Fone[2]+'-'+Fone[3]
                lContato.append(Fone)
            except:
                lContato.append('')

            #ID do Grupo do Contato
            #grupo.append(str(nFileNumber))

            sContatoStatus = 'Saiu'
            if 'entrou' in rowFone:
                sContatoStatus = 'Cadastrado'
            lContatoStatus.append(sContatoStatus)

            # Grupo do contato
            lGrupo.append(nFileNumber)

        dfGrupo = pd.DataFrame(list(zip(lDatas, lContato, lContatoStatus, lGrupo)), columns=['timestamp', 'contato', 'status', 'grupo'])
        dfGrupo['timestamp'] = pd.to_datetime(dfGrupo.timestamp, format='%d/%m/%Y %H:%M')

        # remove events not associated with a sender
        dfGrupo = dfGrupo[dfGrupo.contato != ''].reset_index(drop=True)

        dfTotal = dfTotal.append(dfGrupo)

    return dfTotal

In [122]:
df = parse_file(45)
df

,timestamp,contato,status,grupo
0,2020-12-25 11:41:00,+55 21 97956-9055,Cadastrado,1
1,2020-12-25 11:42:00,+55 35 9157-1445,Cadastrado,1
2,2020-12-25 11:42:00,+55 11 98206-4275,Cadastrado,1
3,2020-12-25 11:42:00,+55 17 98135-0947,Cadastrado,1
4,2020-12-25 11:42:00,+55 47 9188-3757,Cadastrado,1
...,...,...,...,...
182,2021-01-12 18:35:00,+55 35 8807-3708,Cadastrado,38
183,2021-01-12 18:36:00,+55 64 9955-7929,Cadastrado,38
184,2021-01-12 18:37:00,+55 88 9788-5440,Cadastrado,38
185,2021-01-12 18:38:00,+55 32 9949-7969,Cadastrado,38


In [133]:
df.set_index(["status", "contato"]).count(level="status")

,timestamp,grupo
status,,
Cadastrado,8153,8153
Saiu,1887,1887


In [134]:
8153-1887

6266

In [150]:
dfSairam = df.query("status == 'Saiu'")
dfSairam = pd.DataFrame(dfSairam.contato.unique(), columns=['contato'])
dfSairam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1756 entries, 0 to 1755
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   contato  1756 non-null   object
dtypes: object(1)
memory usage: 13.8+ KB


In [125]:
dfCadastrados = df.query("status == 'Cadastrado'")
dfCadastrados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8153 entries, 0 to 186
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  8153 non-null   datetime64[ns]
 1   contato    8153 non-null   object        
 2   status     8153 non-null   object        
 3   grupo      8153 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 318.5+ KB


In [169]:
dfCadastroUnico = pd.DataFrame(dfCadastrados.contato.unique(), columns=['contato'])
dfCadastroUnico

,contato
0,+55 21 97956-9055
1,+55 35 9157-1445
2,+55 11 98206-4275
3,+55 17 98135-0947
4,+55 47 9188-3757
...,...
7860,+55 35 8807-3708
7861,+55 64 9955-7929
7862,+55 88 9788-5440
7863,+55 32 9949-7969


In [173]:
dfContatosAtivos = dfCadastroUnico[~dfCadastroUnico['contato'].isin(dfSairam['contato'])]

In [175]:
dfContatosAtivos.count()

contato    6112
dtype: int64

In [178]:
with pd.ExcelWriter('contatos.xlsx') as writer: 
     dfContatosAtivos['contato'].to_excel(writer, sheet_name='Ativos')
     dfSairam['contato'].to_excel(writer, sheet_name='Desistentes')